In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import geojson
import geopandas as gpd

---
city-industry panel

In [ ]:
# construct city-industry panel for firm entries
## this panel contains all industries

firm_entry_data = pd.read_csv('./raw/工商企业注册数据/各城市历年企业进入数量(分行业).csv')
firm_treated = pd.read_excel('./raw/示范企业认定数据/企业认定时间短面板(不考虑企业资格被取消).xlsx')
firm_treated = firm_treated[~firm_treated['industry'].isna()]

## convert the industry name into code 
industry_table = pd.read_excel('./raw/其他数据/国民经济行业分类表.xlsx')
ind_dict = industry_table.set_index('行业大类名称')['行业大类代码'].to_dict()
def match_ind(industry):
    try:
        return ind_dict[industry]
    except:
        return np.nan
firm_treated['industry'] = firm_treated.industry.apply(match_ind)
firm_treated['industry'] = firm_treated['industry'].astype(int)

## whether each city-industry-year has demonstration firms
grouped_data = pd.DataFrame()
for i_year in range(2005, 2022, 1):
    i_grouped_data = (firm_treated.groupby(['city', 'industry']).agg(treated_count = (i_year, 'sum')).reset_index())
    i_grouped_data['year'] = i_year
    grouped_data = pd.concat([grouped_data, i_grouped_data]).reset_index(drop=True)

grouped_data['key'] = grouped_data['city'] + '-' + grouped_data['industry'].astype(str) + '-' + grouped_data['year'].astype(str)
grouped_data['treated'] = grouped_data['treated_count'].apply(lambda x : 1 if x > 0 else 0)
denum_dict = grouped_data.set_index('key')['treated_count'].to_dict()
treated_dict = grouped_data.set_index('key')['treated'].to_dict()

## Merge into panel dataset
firm_entry_data['key'] = firm_entry_data['city'] + '-' + firm_entry_data['ind'].astype(str) + '-' + firm_entry_data['year'].astype(str)
firm_entry_data['DEnum'] = firm_entry_data['key'].map(denum_dict)
firm_entry_data['treated'] = firm_entry_data['key'].map(treated_dict)
firm_entry_data['DEnum'] = firm_entry_data['DEnum'].fillna(0)
firm_entry_data['treated'] = firm_entry_data['treated'].fillna(0)

firm_entry_data = firm_entry_data.drop('key', axis=1)
firm_entry_data.to_csv('./build/firm_entry_panel.csv', index=False)

In [ ]:
# Construct city-industry level panel (1): Baseline
# Note: This dataset is specifically designed for patent-level analysis, thus only includes 54 major industries that can be matched with patent IPC codes

city_ind_panel = pd.read_csv('./raw/专利数据/各城市历年专利申请数量(分行业).csv')
city_ind_panel = city_ind_panel.rename(columns={
    '实用新型专利数量' : 'utility', 
    '发明公开专利数量' : 'invention', 
    '发明授权专利数量' : 'invention_granted'}
)

firm_treated = pd.read_excel('./raw/示范企业认定数据/企业认定时间短面板(不考虑企业资格被取消).xlsx')
firm_treated = firm_treated[~firm_treated['industry'].isna()]

## Convert the names of major industries of demonstration enterprises into codes
industry_table = pd.read_excel('./raw/其他数据/国民经济行业分类表.xlsx')
ind_dict = industry_table.set_index('行业大类名称')['行业大类代码'].to_dict()
def match_ind(industry):
    try:
        return ind_dict[industry]
    except:
        return np.nan
firm_treated['industry'] = firm_treated.industry.apply(match_ind)
firm_treated['industry'] = firm_treated['industry'].astype(int)

## Calculate whether each city-industry-year has demonstration enterprises
grouped_data = pd.DataFrame()
for i_year in range(2005, 2022, 1):
    i_grouped_data = (firm_treated.groupby(['city', 'industry']).agg(treated_count = (i_year, 'sum')).reset_index())
    i_grouped_data['year'] = i_year
    grouped_data = pd.concat([grouped_data, i_grouped_data]).reset_index(drop=True)

grouped_data['key'] = grouped_data['city'] + '-' + grouped_data['industry'].astype(str) + '-' + grouped_data['year'].astype(str)
grouped_data['treated'] = grouped_data['treated_count'].apply(lambda x : 1 if x > 0 else 0)
denum_dict = grouped_data.set_index('key')['treated_count'].to_dict()
treated_dict = grouped_data.set_index('key')['treated'].to_dict()

## Match to panel
city_ind_panel['key'] = city_ind_panel['city'] + '-' + city_ind_panel['ind'].astype(str) + '-' + city_ind_panel['year'].astype(str)
city_ind_panel['DEnum'] = city_ind_panel['key'].map(denum_dict)
city_ind_panel['treated'] = city_ind_panel['key'].map(treated_dict)
city_ind_panel['DEnum'] = city_ind_panel['DEnum'].fillna(0)
city_ind_panel['treated'] = city_ind_panel['treated'].fillna(0)

## Match industry categories for each industry
ind_data = pd.read_excel('./raw/其他数据/国民经济行业分类表.xlsx')
ind_dict = ind_data.set_index('行业大类代码')['行业门类代码'].to_dict()
city_ind_panel['ind_a'] = city_ind_panel['ind'].map(ind_dict)
ind_a = city_ind_panel.pop('ind_a')
city_ind_panel.insert(4, 'ind_a', ind_a)

city_ind_panel = city_ind_panel.drop('key', axis=1)
city_ind_panel.to_csv('./build/city_ind_panel.csv', index=False)


In [ ]:
# Construct city-industry level panel (2): Incorporating firm entry data

city_ind_panel = pd.read_csv('build/city_ind_panel.csv')
city_ind_panel['index'] = city_ind_panel['prov'] + '-' + city_ind_panel['city'] + '-' + city_ind_panel['year'].astype(str) + '-' + city_ind_panel['ind'].astype(str)

firm_entry_data = pd.read_csv('./raw/工商企业注册数据/各城市历年企业进入数量(分行业).csv')
firm_entry_data['index'] = firm_entry_data['prov'] + '-' + firm_entry_data['city'] + '-' + firm_entry_data['year'].astype(str) + '-' + firm_entry_data['ind'].astype(str)
firm_entry_dict = firm_entry_data.set_index('index')['firm_entry'].to_dict()
state_owned_entry_dict = firm_entry_data.set_index('index')['state_owned_entry'].to_dict()
private_entry_dict = firm_entry_data.set_index('index')['private_entry'].to_dict()
foreign_entry_dict = firm_entry_data.set_index('index')['foreign_entry'].to_dict()
collective_entry_dict = firm_entry_data.set_index('index')['collective_entry'].to_dict()
individual_entry_dict = firm_entry_data.set_index('index')['individual_entry'].to_dict()

city_ind_panel['firm_entry'] = city_ind_panel['index'].apply(lambda x : firm_entry_dict[x])
city_ind_panel['state_owned_entry'] = city_ind_panel['index'].apply(lambda x : state_owned_entry_dict[x])
city_ind_panel['private_entry'] = city_ind_panel['index'].apply(lambda x : private_entry_dict[x])
city_ind_panel['foreign_entry'] = city_ind_panel['index'].apply(lambda x : foreign_entry_dict[x])
city_ind_panel['collective_entry'] = city_ind_panel['index'].apply(lambda x : collective_entry_dict[x])
city_ind_panel['individual_entry'] = city_ind_panel['index'].apply(lambda x : individual_entry_dict[x])

city_ind_panel = city_ind_panel.drop('index', axis=1)
city_ind_panel.to_csv('./build/city_ind_panel.csv', index=False)


In [ ]:
# Construct city-industry level panel (3): Matching weighted patent counts

city_ind_panel = pd.read_csv('build/city_ind_panel.csv')
city_ind_panel['city_year_ind'] = city_ind_panel['city'] + '-' + city_ind_panel['year'].astype(str) + '-' + city_ind_panel['ind'].astype(str)

patent_data = pd.read_csv('./raw/专利数据/各城市历年引用加权专利申请数量(分行业).csv')
invention_adj_3y_dict = patent_data.set_index('city_year_ind')['invention_adj_3y'].to_dict()
invention_adj_5y_dict = patent_data.set_index('city_year_ind')['invention_adj_5y'].to_dict()
city_ind_panel['invention_weighted_3y'] = city_ind_panel['city_year_ind'].map(invention_adj_3y_dict)
city_ind_panel['invention_weighted_5y'] = city_ind_panel['city_year_ind'].map(invention_adj_5y_dict)

city_ind_panel = city_ind_panel.drop('city_year_ind', axis=1)
city_ind_panel.to_csv('./build/city_ind_panel.csv', index=False)


In [ ]:
# Construct city-industry level panel (4): Matching local citation counts

city_ind_panel = pd.read_csv('build/city_ind_panel.csv')
city_ind_panel['city_ind_year'] = city_ind_panel['city'] + '-' + city_ind_panel['ind'].astype(str) + '-' + city_ind_panel['year'].astype(str)

local_citation_data = pd.read_csv('./raw/专利数据/各城市历年专利本地引用数量(分行业).csv')
local_citation_dict = local_citation_data.set_index('city_ind_year')['local_citation'].to_dict()
city_ind_panel['local_citation'] = city_ind_panel['city_ind_year'].map(local_citation_dict)

city_ind_panel = city_ind_panel.drop('city_ind_year', axis=1)
city_ind_panel.to_csv('./build/city_ind_panel.csv', index=False)


In [ ]:
# Determine spillover types for each city-industry pair

city_ind_panel = pd.read_csv('./build/city_ind_panel.csv')
city_ind_panel = city_ind_panel[(city_ind_panel['year'] >= 2008) & (city_ind_panel['year']<=2018)].reset_index(drop=True)

## Calculate treatment year for each city-industry pair using groupby and aggregation
city_ind_panel['city_ind'] = city_ind_panel['city'] + '-' + city_ind_panel['ind'].astype(str)
grouped = city_ind_panel.groupby('city_ind')
city_ind_treated_year = grouped['treated'].sum().apply(lambda x : 2019 - x).to_dict()

city_ind_panel['cohort'] = city_ind_panel['city_ind'].map(city_ind_treated_year)
city_ind_panel['cohort'] = city_ind_panel['cohort'].apply(lambda x : 0 if x == 2019 else x)

## Determine treatment year for each city (inter-industry spillovers)
city_cohort = city_ind_panel.loc[:, ['city', 'cohort']].drop_duplicates().reset_index(drop=True)
city_cohort = city_cohort[city_cohort['cohort']!=0]
city_grouped = city_cohort.groupby('city')
city_cohort_dict = city_grouped['cohort'].min().to_dict()

city_ind_panel['inter_ind_sp_cohort'] = city_ind_panel['city'].map(city_cohort_dict)
city_ind_panel['inter_ind_sp_cohort'] = city_ind_panel['inter_ind_sp_cohort'].fillna(0)
city_ind_panel['inter_ind_sp_cohort'] = city_ind_panel.apply(
        lambda row : np.nan if row['cohort']!=0 else row['inter_ind_sp_cohort'],
    axis=1) # Set treated group values to np.nan directly

## Determine treatment year for each industry (spatial spillovers)
ind_cohort = city_ind_panel.loc[:, ['ind', 'cohort']].drop_duplicates().reset_index(drop=True)
ind_cohort = ind_cohort[ind_cohort['cohort']!=0]
ind_grouped = ind_cohort.groupby('ind')
ind_cohort_dict = ind_grouped['cohort'].min().to_dict()

city_ind_panel['spatial_sp_cohort'] = city_ind_panel['ind'].map(ind_cohort_dict)
city_ind_panel['spatial_sp_cohort'] = city_ind_panel['spatial_sp_cohort'].fillna(0)
city_ind_panel['spatial_sp_cohort'] = city_ind_panel.apply(
        lambda row : np.nan if row['cohort']!=0 else row['spatial_sp_cohort'],
    axis=1) # Set treated group values to np.nan directly

## Remove original treatment group samples, and distinguish among control groups:
## "pure controls", "spatial spillover controls", "inter-industry spillover controls", and "mixed spillover controls"
def identify_spillover_type(inter_ind_sp_cohort, spatial_sp_cohort):
    if inter_ind_sp_cohort == 0 and spatial_sp_cohort == 0:
        return 'pure_controls'
    elif inter_ind_sp_cohort != 0 and spatial_sp_cohort == 0:
        return 'inter_industry_spillovers'
    elif inter_ind_sp_cohort == 0 and spatial_sp_cohort != 0:
        return 'spatial_spillovers'
    elif inter_ind_sp_cohort != 0 and spatial_sp_cohort != 0:
        return 'mixed_spillovers'

spillover_panel = city_ind_panel[city_ind_panel['cohort']==0].reset_index(drop=True)
spillover_panel['spillover_type'] = spillover_panel.apply(
        lambda row : identify_spillover_type(row['inter_ind_sp_cohort'], row['spatial_sp_cohort']),
    axis=1)

spillover_panel = spillover_panel.drop('city_ind', axis=1)
spillover_panel.to_csv('./build/spillover_panel.csv', index=False)


---
city-industry stacked panel

In [ ]:
# Construct city-industry stacked panel (firm entry panel)

firm_entry_panel = pd.read_csv('./build/firm_entry_panel.csv')
firm_entry_panel['year'].value_counts()

## Calculate treatment year for each city-industry pair using groupby and aggregation
firm_entry_panel['city_ind'] = firm_entry_panel['city'] + '-' + firm_entry_panel['ind'].astype(str)
grouped = firm_entry_panel.groupby('city_ind')
city_ind_treated_year = grouped['treated'].sum().apply(lambda x : 2021 - x).to_dict()

firm_entry_panel['cohort'] = firm_entry_panel['city_ind'].map(city_ind_treated_year)
firm_entry_panel['cohort'] = firm_entry_panel['cohort'].apply(lambda x : 0 if x == 2021 else x)

## Generate control groups for each treatment cohort
stacked_panel = pd.DataFrame()
cohort_list = sorted([i for i in list(firm_entry_panel['cohort'].drop_duplicates()) if i!=0])
for each_cohort in cohort_list:
    each_block = firm_entry_panel[(firm_entry_panel['cohort']==each_cohort) | (firm_entry_panel['cohort']==0)].copy()
    each_block['block'] = each_cohort
    stacked_panel = pd.concat([stacked_panel, each_block], axis=0)

stacked_panel = stacked_panel.reset_index(drop=True)
stacked_panel.to_csv('./build/firm_entry_stacked_panel.csv', index=False)


In [ ]:
# Construct city-industry stacked panel (patent panel)

city_ind_panel = pd.read_csv('./build/city_ind_panel.csv')

## Calculate treatment year for each city-industry pair using groupby and aggregation
city_ind_panel['city_ind'] = city_ind_panel['city'] + '-' + city_ind_panel['ind'].astype(str)
grouped = city_ind_panel.groupby('city_ind')
city_ind_treated_year = grouped['treated'].sum().apply(lambda x : 2021 - x).to_dict()

city_ind_panel['cohort'] = city_ind_panel['city_ind'].map(city_ind_treated_year)
city_ind_panel['cohort'] = city_ind_panel['cohort'].apply(lambda x : 0 if x == 2021 else x)

## Generate control groups for each treatment cohort
stacked_panel = pd.DataFrame()
cohort_list = sorted([i for i in list(city_ind_panel['cohort'].drop_duplicates()) if i!=0])
for each_cohort in cohort_list:
    each_block = city_ind_panel[(city_ind_panel['cohort']==each_cohort) | (city_ind_panel['cohort']==0)].copy()
    each_block['block'] = each_cohort
    stacked_panel = pd.concat([stacked_panel, each_block], axis=0)

stacked_panel = stacked_panel.reset_index(drop=True)

stacked_panel.to_csv('./build/city_ind_stacked_panel.csv', index=False)


In [ ]:
# Construct stacked panel for spatial spillovers (patent panel)

sp_panel = pd.read_csv('./build/spillover_panel.csv')
spatial_sp = sp_panel[(sp_panel['spillover_type'] == 'pure_controls') | (sp_panel['spillover_type'] == 'spatial_spillovers')]

## Generate control groups for each treatment cohort
spatial_sp_stacked_panel = pd.DataFrame()
cohort_list = sorted([i for i in list(spatial_sp['spatial_sp_cohort'].drop_duplicates()) if i!=0])
for each_cohort in cohort_list:
    each_block = spatial_sp[(spatial_sp['spatial_sp_cohort']==each_cohort) | (spatial_sp['spatial_sp_cohort']==0)].copy()
    each_block['block'] = each_cohort
    spatial_sp_stacked_panel = pd.concat([spatial_sp_stacked_panel, each_block], axis=0)
    
spatial_sp_stacked_panel.to_csv('./build/spillover_stacked_panel_spatial.csv', index=False)


In [ ]:
# Construct stacked panel for inter-industry spillovers (patent panel)

sp_panel = pd.read_csv('./build/spillover_panel.csv')
ind_sp = sp_panel[(sp_panel['spillover_type'] == 'pure_controls') | (sp_panel['spillover_type'] == 'inter_industry_spillovers')]

## Generate control groups for each treatment cohort
inter_ind_sp_stacked_panel = pd.DataFrame()
cohort_list = sorted([i for i in list(ind_sp['inter_ind_sp_cohort'].drop_duplicates()) if i!=0])
for each_cohort in cohort_list:
    each_block = ind_sp[(ind_sp['inter_ind_sp_cohort']==each_cohort) | (ind_sp['inter_ind_sp_cohort']==0)].copy()
    each_block['block'] = each_cohort
    inter_ind_sp_stacked_panel = pd.concat([inter_ind_sp_stacked_panel, each_block], axis=0)

inter_ind_sp_stacked_panel.to_csv('./build/spillover_stacked_panel_inter_ind.csv', index=False)


---
estimating spatial spillovers using adjacent cities

In [ ]:
# Construct city adjacency matrix using map data

city_ind_panel = pd.read_csv('./build/city_ind_panel.csv')
city_ind_panel = city_ind_panel[(city_ind_panel['year'] >= 2008) & (city_ind_panel['year']<=2018)].reset_index(drop=True)
city_list = city_ind_panel['city'].drop_duplicates().to_list()

with open('./raw/其他数据/city_map.json', 'r', encoding='utf-8') as map_file:
    data = geojson.load(map_file)

city_gdf = gpd.GeoDataFrame.from_features(data['features'])

## Process map data - merge districts of Beijing, Tianjin, Shanghai, and Chongqing into single cities
code_name_dict = {'110' : '北京市', '120' : '天津市', '310' : '上海市', '500' : '重庆市'}
province_codes = ['110', '120', '310', '500']
gdf_provinces = city_gdf[city_gdf['adcode'].astype(str).str.startswith(tuple(province_codes))]
merged_gdf = []

for code in province_codes:
    province_gdf = gdf_provinces[gdf_provinces['adcode'].astype(str).str.startswith(code)]
    merged_geometry = province_gdf.geometry.unary_union

    first_row = province_gdf.iloc[0]
    merged_row = first_row.copy()
    merged_row['geometry'] = merged_geometry
    merged_row['adcode'] = code
    merged_row['name'] = code_name_dict[code]

    merged_gdf.append(merged_row)

merged_gdf = gpd.GeoDataFrame(merged_gdf, crs=city_gdf.crs)

city_gdf = city_gdf[~city_gdf['adcode'].astype(str).str.startswith(tuple(province_codes))] # Remove district-level data for the four municipalities
city_gdf = city_gdf.append(merged_gdf, ignore_index=True) # Add merged municipality data
city_gdf = city_gdf[city_gdf['name'].isin(city_list)] # Keep only cities present in the city-industry panel

city_gdf['is_valid'] = city_gdf.is_valid
city_gdf['geometry'] = city_gdf['geometry'].apply(lambda geom: geom.buffer(0) if not geom.is_valid else geom)
city_gdf['is_valid'] = city_gdf.is_valid

## Construct city adjacency table
n = len(city_gdf)
city_pairs_list = []

for i in range(n):
    for j in range(i + 1, n): 
        city_1 = city_gdf.iloc[i]
        city_2 = city_gdf.iloc[j]
        
        is_adjacent = 1 if city_1.geometry.intersects(city_2.geometry) else 0
        city_pairs_list.append((city_1['name'], city_2['name'], is_adjacent)) 

city_pairs = pd.DataFrame(city_pairs_list, columns=['city_1', 'city_2', 'is_adjacent'])

## Convert to dictionary for easier matching
adjacent_dict = {}

for i_city in city_list:
    i_matched_data = city_pairs[
            (
                (city_pairs['city_1']==i_city) | 
                (city_pairs['city_2']==i_city)
            ) & 
            (city_pairs['is_adjacent']==1)
        ]
    i_adjacent_city = list(set(i_matched_data['city_1'].to_list() + i_matched_data['city_2'].to_list()))
    
    if len(i_adjacent_city) > 0:
        i_adjacent_city.remove(i_city)
    
    adjacent_dict.update({i_city : i_adjacent_city})


In [ ]:
# Construct firm-industry panel data

## Calculate treatment year for each city-industry pair using groupby and aggregation
city_ind_panel['city_ind'] = city_ind_panel['city'] + '-' + city_ind_panel['ind'].astype(str)
grouped = city_ind_panel.groupby('city_ind')
city_ind_treated_year = grouped['treated'].sum().apply(lambda x : 2019 - x).to_dict()

city_ind_panel['cohort'] = city_ind_panel['city_ind'].map(city_ind_treated_year)
city_ind_panel['cohort'] = city_ind_panel['cohort'].apply(lambda x : 0 if x == 2019 else x)

treated_cohort = city_ind_panel.loc[:, ['city', 'ind', 'cohort']].drop_duplicates().reset_index(drop=True)
treated_cohort = treated_cohort[treated_cohort['cohort']!=0].reset_index(drop=True)
treated_cohort['adjacent_city'] = treated_cohort['city'].map(adjacent_dict) ## Map adjacent cities for each city

## Identify control group firms that may have spatial spillover effects
spillover_panel = city_ind_panel[city_ind_panel['cohort']==0].reset_index(drop=True)
spillover_panel['spatial_sp_cohort'] = np.nan

for index, row in treated_cohort.iterrows():
    i_city = treated_cohort.loc[index, 'city']
    i_ind = treated_cohort.loc[index, 'ind']
    i_cohort = treated_cohort.loc[index, 'cohort']
    i_adjacent_city = treated_cohort.loc[index, 'adjacent_city']

    if len(i_adjacent_city) > 0:
        for j_city in i_adjacent_city:
            j_city_cohort = spillover_panel.loc[(spillover_panel['city']==j_city) & (spillover_panel['ind']==i_ind), 'spatial_sp_cohort'].mean() # Check if value already exists
            if np.isnan(j_city_cohort): # If no value exists, assign current cohort
                spillover_panel.loc[(spillover_panel['city']==j_city) & (spillover_panel['ind']==i_ind), 'spatial_sp_cohort'] = i_cohort
            else: # If value exists, keep the earlier cohort
                if j_city_cohort > i_cohort: # If existing value is later, replace with current cohort
                    spillover_panel.loc[(spillover_panel['city']==j_city) & (spillover_panel['ind']==i_ind), 'spatial_sp_cohort'] = i_cohort

spillover_panel['spatial_sp_cohort'] = spillover_panel['spatial_sp_cohort'].fillna(0)
spillover_panel.to_csv('./build/spillover_panel_adjacent.csv', index=False)


In [ ]:
# Construct stacked panel for spatial spillover analysis

spatial_sp = pd.read_csv('./build/spillover_panel_adjacent.csv')

## Generate control groups for each treatment cohort
spatial_sp_stacked_panel = pd.DataFrame()
cohort_list = sorted([i for i in list(spatial_sp['spatial_sp_cohort'].drop_duplicates()) if i!=0])
for each_cohort in cohort_list:
    each_block = spatial_sp[(spatial_sp['spatial_sp_cohort']==each_cohort) | (spatial_sp['spatial_sp_cohort']==0)].copy()
    each_block['block'] = each_cohort
    spatial_sp_stacked_panel = pd.concat([spatial_sp_stacked_panel, each_block], axis=0)
    
spatial_sp_stacked_panel.to_csv('./build/spillover_stacked_panel_spatial_adjacent.csv', index=False)


---
listed firm panel

In [ ]:
# Construct listed firm panel (1): Basic information

listed_firm = pd.read_excel('./raw/上市公司数据/上市公司信息(CSMAR).xlsx', usecols=[0, 2, 8, 24, 26, 34, 36, 38])
listed_firm.head(1)

## Process basic information
listed_firm['stock_code'] = listed_firm['股票代码'].apply(lambda x: '0' * (6 - len(str(x))) + str(x))
listed_firm['year'] = listed_firm['统计截止日期'].apply(lambda x: int(x[:4]))
listed_firm['ind'] = listed_firm['行业代码C'].apply(lambda x: x[-2:])
listed_firm['registered_capital'] = listed_firm['注册资本'].apply(lambda x: int(x))
listed_firm['established_year'] = listed_firm['公司成立日期'].apply(lambda x: int(x[:4]) if type(x)==str else '')
listed_firm = listed_firm[(listed_firm['year']>=2008) & (listed_firm['year']<=2018)].copy().reset_index(drop=True)
listed_firm = listed_firm.rename(columns={'所属省份': 'prov', '所属城市': 'city'})

## Remove firms with non-normal listing status during sample period (2008-2015)
dropped_firm = listed_firm.loc[:, ['上市状态', 'stock_code']].copy().drop_duplicates()
dropped_firm['times'] = dropped_firm['stock_code'].apply(lambda x: list(dropped_firm['stock_code']).count(x))
keeped_firm = list(dropped_firm[(dropped_firm['times']==1) & (dropped_firm['上市状态']=='正常上市')].stock_code)
listed_firm = listed_firm[listed_firm['stock_code'].isin(keeped_firm)].reset_index(drop=True)

## Handle industry code changes (set each firm's industry to its mode during sample period)
listed_firm['year'].value_counts()
stock_code_list = listed_firm['stock_code'].drop_duplicates().to_list()
for i_code in stock_code_list:
    code_ind = listed_firm.loc[listed_firm['stock_code']==i_code, 'ind'].values.astype(int)
    ind_mode = stats.mode(code_ind, keepdims=False)[0]
    listed_firm.loc[listed_firm['stock_code']==i_code, 'ind'] = ind_mode

listed_firm = listed_firm.reindex(columns=['stock_code', 'year', 'ind', 'prov', 'city', 'registered_capital', 'established_year'])


In [ ]:
# Building listed firms panel (2): Matching patent counts

## First process patent data
patent_data = pd.read_excel('./raw/上市公司数据/上市公司专利申请情况(CNRDS).xlsx')

patent_data['stock_code'] = patent_data['股票代码'].apply(lambda x : '0' * (6 - len(str(x))) + str(x))
patent_data = patent_data[patent_data['公司类型']=='集团公司合计'].reset_index(drop=True)
patent_data['invention'] = patent_data['当年独立申请的发明数量'] + patent_data['当年联合申请的发明数量']
patent_data['utility'] = patent_data['当年独立申请的实用新型数量'] + patent_data['当年联合申请的实用新型数量']
patent_data['patent'] = patent_data['invention'] + patent_data['utility']
patent_data['year'] = patent_data['会计年度'].apply(int)

patent_data = patent_data.reindex(columns=['stock_code', 'year', 'invention', 'utility', 'patent'])
patent_data['code_year'] = patent_data['stock_code'].astype(str) + '-' + patent_data['year'].astype(str)
invention_dict = patent_data.set_index('code_year')['invention'].to_dict()
utility_dict = patent_data.set_index('code_year')['utility'].to_dict()
patent_dict = patent_data.set_index('code_year')['patent'].to_dict()

## Matching
listed_firm['index'] = listed_firm['stock_code'].astype(str) + '-' + listed_firm['year'].astype(str)
listed_firm['utility'] = listed_firm['index'].map(utility_dict)
listed_firm['invention'] = listed_firm['index'].map(invention_dict)

listed_firm = listed_firm.drop('index', axis=1)


In [ ]:
# Construct listed firms panel (3): Matching demonstration firms and identifying competitor firms

## Matching demonstration firms
demon_firm = pd.read_excel('./raw/示范企业认定数据/示范企业信息.xlsx')
demon_firm = demon_firm[~demon_firm['stock_code'].isna()].reset_index(drop=True)
demon_firm['stock_code'] = demon_firm['stock_code'].apply(lambda x : '0' * (6 - len(str(int(x)))) + str(int(x)))

demon_year = pd.read_excel('./raw/示范企业认定数据/企业认定时间短面板(不考虑企业资格被取消).xlsx')
demon_year['demon_year'] = np.nan
for index, row in demon_year.iterrows():
    demon_year.loc[index, 'demon_year'] = list(demon_year.iloc[index, 4:22]).index(1) + 2005
demon_year['demon_year'] = demon_year['demon_year'].astype(int)
demon_year_dict = demon_year.set_index('企业名称')['demon_year'].to_dict()
demon_firm['demon_year'] = demon_firm['each_firm'].apply(lambda x : demon_year_dict[x])
code_year_dict = demon_firm[demon_firm['demon_year']<=2018].set_index('stock_code')['demon_year'].to_dict()

demon_firm_list = list(demon_firm[demon_firm['demon_year']<=2018]['stock_code']) ## Generate list of demonstration firms before 2018

listed_firm['demon_firm'] = listed_firm['stock_code'].apply(lambda x : 1 if x in demon_firm_list else 0) # Identify whether the firm is a demonstration firm
listed_firm['treatment_year'] = listed_firm['stock_code'].map(code_year_dict) # Match treatment year for demonstration firms

listed_firm['treated'] = np.nan
for index, row in listed_firm.iterrows():
    if listed_firm.loc[index, 'demon_firm'] == 0:
        listed_firm.loc[index, 'treated'] = 0
    elif listed_firm.loc[index, 'treatment_year'] > listed_firm.loc[index, 'year']:
        listed_firm.loc[index, 'treated'] = 0
    elif listed_firm.loc[index, 'treatment_year'] <= listed_firm.loc[index, 'year']:
        listed_firm.loc[index, 'treated'] = 1

## Identifying competitor firms
demon_city_ind = listed_firm[listed_firm['demon_firm']==1].loc[:, ['city', 'ind', 'treatment_year']].drop_duplicates()
demon_city_ind['city_ind'] = demon_city_ind['city'] + '-' + demon_city_ind['ind'].astype(str)
demon_city_ind_list = demon_city_ind['city_ind'].to_list()

demon_city_ind['earliest_treatment_year'] = demon_city_ind.apply(
        lambda row : demon_city_ind[(demon_city_ind['city']==row['city']) & (demon_city_ind['ind']==row['ind'])]
            ['treatment_year'].min(), 
    axis=1) # Use the first year when a demonstration firm appears as the treatment year for competitors
earliest_year_dict = demon_city_ind.set_index('city_ind')['earliest_treatment_year'].to_dict()

listed_firm['competitor_firm'] = listed_firm.apply(
        lambda row : 1 if ((row['city'] + '-' + row['ind'].astype(str)) in demon_city_ind_list) and (row['demon_firm']==0) else 0, 
    axis=1) # Identify whether the firm is a competitor firm

listed_firm.loc[listed_firm['competitor_firm']==1, 'treatment_year'] = listed_firm[listed_firm['competitor_firm']==1].apply(
        lambda row : earliest_year_dict[row['city'] + '-' + row['ind'].astype(str)], 
    axis=1) # Match treatment year for competitor firms

listed_firm.to_csv('./build/listed_panel.csv', index=False)


In [ ]:
# Construct listed firms panel (4): Matching financial information

listed_firm = pd.read_csv('./build/listed_panel.csv')
listed_firm['code_year'] = listed_firm['stock_code'].astype(str) + listed_firm['year'].astype(str)

asset_info = pd.read_excel('./raw/上市公司数据/上市公司财务数据(CSMAR).xlsx')
asset_info['code_year'] = asset_info['证券代码'].astype(str) + asset_info['年份'].astype(str)

asset_dict = asset_info.set_index('code_year')['资产总计'].to_dict()
rd_dict = asset_info.set_index('code_year')['研发费用'].to_dict()
revenue_dict = asset_info.set_index('code_year')['营业收入'].to_dict() 
profit_dict = asset_info.set_index('code_year')['营业利润'].to_dict()
wage_dict = asset_info.set_index('code_year')['支付给职工以及为职工支付的现金'].to_dict()

listed_firm['asset'] = listed_firm['code_year'].map(asset_dict)
listed_firm['rd'] = listed_firm['code_year'].map(rd_dict)
listed_firm['revenue'] = listed_firm['code_year'].map(revenue_dict)
listed_firm['profit'] = listed_firm['code_year'].map(profit_dict)
listed_firm['wage'] = listed_firm['code_year'].map(wage_dict)

listed_firm = listed_firm.drop(['code_year'], axis=1)
listed_firm.to_csv('./build/listed_panel.csv', index=False)


In [ ]:
# Construct listed firms panel (5): Matching R&D expenditure data

listed_firm = pd.read_csv('./build/listed_panel.csv')
listed_firm['code_year'] = listed_firm['stock_code'].astype(str) + '-' + listed_firm['year'].astype(str)

rd_exp = pd.read_excel('./raw/上市公司数据/上市公司研发支出(CNRDS).xlsx')
rd_exp['code_year'] = rd_exp['股票代码'].astype(str) + '-' + rd_exp['会计年度'].astype(str)
rd_dict = rd_exp.set_index('code_year')['研发支出'].to_dict()
rd_share_dict = rd_exp.set_index('code_year')['研发支出占营业收入比例'].to_dict()
rd_worker_dict = rd_exp.set_index('code_year')['研发人员数量'].to_dict()
rd_worker_share_dict = rd_exp.set_index('code_year')['研发人员数量占比'].to_dict()

listed_firm['rd'] = listed_firm['code_year'].map(rd_dict)
listed_firm['rd_share'] = listed_firm['code_year'].map(rd_share_dict)
listed_firm['rd_worker'] = listed_firm['code_year'].map(rd_worker_dict)
listed_firm['rd_worker_share'] = listed_firm['code_year'].map(rd_worker_share_dict)

listed_firm = listed_firm.drop(['code_year'], axis=1)
listed_firm.to_csv('./build/listed_panel.csv', index=False)


In [ ]:
# Construct listed firms panel (6): Matching government subsidy data

listed_firm = pd.read_csv('./build/listed_panel.csv')
listed_firm['code_year'] = listed_firm['stock_code'].astype(str) + '-' + listed_firm['year'].astype(str)

subsidy_data = pd.read_excel('./raw/上市公司数据/上市公司政府补贴(CSMAR).xlsx')
subsidy_data['year'] = subsidy_data['统计截止日期'].apply(lambda x : int(x[:4]))
subsidy_data['code_year'] = subsidy_data['证券代码'].astype(str) + '-' + subsidy_data['year'].astype(str)
subsidy_data = subsidy_data[subsidy_data['项目']!='合计'].reset_index(drop=True) # Remove aggregate items

## Set keywords to identify innovation-related subsidies
keywords_list = ['创新', '研究', '研发', '科研', '专利', '人才', '科技', '技术']
subsidy_data['innovation_subsidy'] = subsidy_data['项目'].apply(
            lambda x : 1 if any(keyword in str(x) for keyword in keywords_list) else 0
        )
innovation_proportion = subsidy_data['innovation_subsidy'].sum() / len(subsidy_data)
print('R&D subsidy proportion: ' + str(round(innovation_proportion, 3)))

subsidy_data = subsidy_data[subsidy_data['innovation_subsidy']==1]
grouped_data = subsidy_data.groupby(['code_year']).agg(total_subsidy = ('本期金额', 'sum')).reset_index()
subsidy_dict = grouped_data.set_index('code_year')['total_subsidy'].to_dict()

## Match to listed firms data
listed_firm['code_year'] = listed_firm['stock_code'].astype(str) + '-' + listed_firm['year'].astype(str)
listed_firm['subsidy'] = listed_firm['code_year'].map(subsidy_dict)
listed_firm['subsidy'] = listed_firm['subsidy'].fillna(0)

listed_firm = listed_firm.drop(['code_year'], axis=1)
listed_firm.to_csv('./build/listed_panel.csv', index=False)


In [ ]:
# Construct listed firms panel (7): Matching total factor productivity and calculating TFP dispersion at city-industry level

listed_firm = pd.read_csv('./build/listed_panel.csv')
listed_firm['code_year'] = listed_firm['stock_code'].astype(str) + '-' + listed_firm['year'].astype(str)

tfp_op_data = pd.read_excel('./raw/上市公司数据/企业全要素生产率（OP法）.xlsx')
tfp_op_data['code_year'] = tfp_op_data['证券代码'].astype(str) + '-' + tfp_op_data['year'].astype(str)
tfp_op_dict = tfp_op_data.set_index('code_year')['全要素生产率'].to_dict()

listed_firm['tfp_op'] = listed_firm['code_year'].map(tfp_op_dict)

## Calculate Gini coefficient
def calculate_gini(series):
    sorted_series = series.dropna().sort_values()
    if len(sorted_series) > 1:
        n = len(sorted_series)
        total = sorted_series.sum()
        index = np.arange(1, n + 1)
        weighted_sum = (sorted_series * (2 * index - n - 1)).sum()
        
        gini_coefficient = weighted_sum / (n * total)
        return gini_coefficient
    else:
        return np.nan

gini_op_dict = listed_firm.groupby(['city', 'ind', 'year'])['tfp_op'].apply(calculate_gini)
listed_firm['gini_op'] = listed_firm.apply(
    lambda row: gini_op_dict.get((row['city'], row['ind'], row['year']), np.nan), axis=1
)

listed_firm = listed_firm.drop(['code_year'], axis=1)
listed_firm.to_csv('./build/listed_panel.csv', index=False)


---
listed firm stacked panel

In [ ]:
# construct listed firm stack panel (dmonstration firms)

listed_panel = pd.read_csv('./build/listed_panel.csv')
listed_panel_demon = listed_panel[listed_panel['competitor_firm']==0].reset_index(drop=True)

stacked_panel_demon = pd.DataFrame()
cohort_list = sorted([int(i) for i in list(listed_panel['treatment_year'].drop_duplicates()) if np.isnan(i)==False])
for each_cohort in cohort_list:
    each_stack = listed_panel_demon[
                        (listed_panel_demon['treatment_year']==each_cohort) | 
                        (listed_panel_demon['demon_firm']==0)
                    ].copy()
    each_stack['block'] = each_cohort
    stacked_panel_demon = pd.concat([stacked_panel_demon, each_stack], axis=0)

stacked_panel_demon = stacked_panel_demon.reset_index(drop=True)
stacked_panel_demon.to_csv('./build/listed_stacked_panel_demon.csv', index=False)

In [ ]:
# construct listed firm stack panel (competitor firms)

listed_panel = pd.read_csv('./build/listed_panel.csv')
listed_panel_competitor = listed_panel[listed_panel['demon_firm']==0].reset_index(drop=True)

stacked_panel_competitor = pd.DataFrame()
cohort_list = sorted([int(i) for i in list(listed_panel_competitor['treatment_year'].drop_duplicates()) if np.isnan(i)==False])
for each_cohort in cohort_list:
    each_stack = listed_panel_competitor[
                        (listed_panel_competitor['treatment_year']==each_cohort) | 
                        (listed_panel_competitor['competitor_firm']==0)
                    ].copy()
    each_stack['block'] = each_cohort
    stacked_panel_competitor = pd.concat([stacked_panel_competitor, each_stack], axis=0)

stacked_panel_competitor = stacked_panel_competitor.reset_index(drop=True)
stacked_panel_competitor.to_csv('./build/listed_stacked_panel_competitor.csv', index=False)

---
industry-level characteristics

In [ ]:
# Calculate the dispersion of total factor productivity across industries for listed firms

firm_data = pd.read_excel('./raw/上市公司数据/上市公司信息(CSMAR).xlsx', usecols=[0, 2, 8])
firm_data['year'] = firm_data['统计截止日期'].apply(lambda x : int(x[:4]))
firm_data = firm_data[(firm_data['year']>=2008) & (firm_data['year']<=2011)].reset_index(drop=True)
firm_data['ind'] = firm_data['行业代码C'].apply(lambda x : x[1:])
firm_data['key'] = firm_data['股票代码'].astype(str) + '-' + firm_data['year'].astype(str)
firm_ind_dict = firm_data.set_index('key')['ind'].to_dict()

tfp_data = pd.read_excel('./raw/上市公司数据/企业全要素生产率（OP法）.xlsx')
tfp_data = tfp_data[(tfp_data['year']>=2008) & (tfp_data['year']<=2011)].reset_index(drop=True)
tfp_data['key'] = tfp_data['证券代码'].astype(str) + '-' + tfp_data['year'].astype(str)
tfp_data['ind'] = tfp_data['key'].map(firm_ind_dict)

## Calculate annual Gini coefficients of TFP by industry
def calculate_gini(series):
    sorted_series = series.dropna().sort_values()
    if len(sorted_series) > 1:
        n = len(sorted_series)
        total = sorted_series.sum()
        index = np.arange(1, n + 1)
        weighted_sum = (sorted_series * (2 * index - n - 1)).sum()
        
        gini_coefficient = weighted_sum / (n * total)
        return gini_coefficient
    else:
        return np.nan

ind_year_data = (
    tfp_data.groupby(['year', 'ind'])
    .agg(
        firm_num = ('证券代码', 'size'),
        tfp_gini = ('全要素生产率', lambda x : calculate_gini(x))
    )
    .reset_index()
)

## Use annual averages as measures of industry productivity dispersion
ind_data = (
    ind_year_data.groupby('ind')
    .agg(
        mean_tfp_gini = ('tfp_gini', 'mean'),
        mean_firm_num = ('firm_num', 'mean')
    )
    .reset_index()
)
ind_data


In [ ]:
# calculate the number of firm entries in each industry

entry_data = pd.read_csv('./raw/工商企业注册数据/各城市历年企业进入数量(分行业).csv')
entry_data = entry_data[(entry_data['year']>=2008) & (entry_data['year']<=2011)].reset_index(drop=True)
ind_data = entry_data.groupby('ind').agg(mean_entry = ('firm_entry', 'mean')).reset_index()
ind_data